In [ ]:
import tensorflow as tf
from keras.metrics import MeanAbsoluteError
import keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
import pandas as pd
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from keras.layers import Input
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import VGG16, ResNet50, InceptionV3, Xception
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
!pip install keras-tuner
from kerastuner.tuners import RandomSearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 8.3 MB/s eta 0:00:00


<ipython-input-1-a54b6937157a>:26: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:


Dataset = pd.read_csv("/content/Materialdata.csv")
x_set=Dataset.iloc[:,1:-1].values
y_set=Dataset.iloc[:,-1].values


In [ ]:
# Define the number of components to keep after PCA
n_components = 15

# Create a PCA object with the specified number of components
pca = PCA(n_components=n_components)

# Fit the PCA model to the input data and transform the data
X_pca = pca.fit_transform(x_set)

In [ ]:
X_norm = MinMaxScaler(feature_range=(-0.5, 0.5))
Y_norm = MinMaxScaler(feature_range=(-0.99, 0.99))
y_set = y_set.reshape(-1,1)
# Normalize input and target variables

X_pca[:,:] = X_norm.fit_transform(X_pca[:,:])
y_scaled = Y_norm.fit_transform(y_set)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, y_scaled, test_size=1/3, random_state=42)

DNN UNTRAINED WITH PCA

In [ ]:
input_shape=(15,)
Output_shape = 1

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, activation='tanh', input_shape=input_shape),
    tf.keras.layers.Dense(5, activation='tanh'),
    tf.keras.layers.Dense(3, activation='tanh'),
    tf.keras.layers.Dense(2, activation='tanh'),
    tf.keras.layers.Dense(Output_shape , activation='linear')
])

optimizer = tf.keras.optimizers.Adam()

# Compile the model
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mean_absolute_percentage_error'])


model.summary()

In [ ]:
history=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1000, batch_size=32)

In [ ]:
# Plot the training and validation loss
plt.plot(history.history['mean_absolute_error'], label='mean_absolute_error')
plt.plot(history.history['val_mean_absolute_error'], label='val_mean_absolute_error')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

DNN TRAINED MODEL WITH PCA

In [ ]:
# Define the Autoencoder architecture
input_layer = Input(shape=(15,))
encoded_layer1 = Dense(6, activation='tanh')(input_layer)
encoded_layer2 = Dense(5, activation='tanh')(encoded_layer1)
encoded_layer3 = Dense(4, activation='tanh')(encoded_layer2)
encoded_layer4 = Dense(3, activation='tanh')(encoded_layer3)
decoded_layer1 = Dense(4, activation='tanh')(encoded_layer4)
decoded_layer2 = Dense(5, activation='tanh')(decoded_layer1)
decoded_layer3 = Dense(6, activation='tanh')(decoded_layer2)
decoded_layer4 = Dense(15, activation='linear')(decoded_layer3)

autoencoder = Model(input_layer, decoded_layer4)

autoencoder.compile(optimizer='adam', loss='mse', metrics=[MeanAbsolutePercentageError(), 'mape'])
history = autoencoder.fit(x_train, x_train, epochs=1000, batch_size=32, shuffle=True, validation_data=(x_test, x_test))

# Create the Encoder model
encoder = Model(input_layer, encoded_layer4)

# Freeze the Encoder layers
encoder.trainable = False

# Create the pre-trained model
pretrained_model = Model(input_layer, encoder(input_layer))

# Create the final autoencoder
input_layer_final = Input(shape=(15,))
encoded_layer1_final = Dense(6, activation='tanh')(input_layer_final)
encoded_layer2_final = Dense(5, activation='tanh')(encoded_layer1_final)
encoded_layer3_final = Dense(4, activation='tanh')(encoded_layer2_final)
encoded_layer4_final = Dense(3, activation='tanh')(encoded_layer3_final)
decoded_layer1_final = Dense(4, activation='tanh')(encoded_layer4_final)
decoded_layer2_final = Dense(5, activation='tanh')(decoded_layer1_final)
decoded_layer3_final = Dense(6, activation='tanh')(decoded_layer2_final)
decoded_layer4_final = Dense(15, activation='linear')(decoded_layer3_final)

autoencoder_final = Model(input_layer_final, decoded_layer4_final)
autoencoder_final.compile(optimizer='adam', loss='mse', metrics=['mape'])
history_final = autoencoder_final.fit(x_train, x_train, epochs=1000, batch_size=32, shuffle=True, validation_data=(x_test, x_test))

# Create the Encoder model for the final autoencoder
encoder_final = Model(input_layer_final, encoded_layer4_final)

# Freeze the final autoencoder Encoder layers
encoder_final.trainable = False

# Create the pre-trained model for the final autoencoder
pretrained_model_final = Model(input_layer_final, encoder_final(input_layer_final))

# Extract the hidden layer output of the final autoencoder
hidden_layer_output_final = encoder_final.predict(x_train)


In [ ]:
plt.plot(history.history['mean_absolute_error'], label='mean_absolute_error')
plt.plot(history.history['val_mean_absolute_error'], label='Validation_mean_absolute_error')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.35])
plt.legend()
plt.show()


In [ ]:
X_norm = MinMaxScaler(feature_range=(-0.5, 0.5))
Y_norm = MinMaxScaler(feature_range=(-0.99, 0.99))
y_set = y_set.reshape(-1,1)

x_set[:,:] = X_norm.fit_transform(x_set[:,:])
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_set, y_scaled, test_size=1/3, random_state=42)

In [ ]:
# Define the first Autoencoder
input_layer1 = Input(shape=(21,))
encoded_layer1 = Dense(6, activation='tanh')(input_layer1)
encoded_layer2 = Dense(5, activation='tanh')(encoded_layer1)
encoded_layer3 = Dense(4, activation='tanh')(encoded_layer2)
encoded_layer4 = Dense(3, activation='tanh')(encoded_layer3)
decoded_layer1 = Dense(4, activation='tanh')(encoded_layer4)
decoded_layer2 = Dense(5, activation='tanh')(decoded_layer1)
decoded_layer3 = Dense(6, activation='tanh')(decoded_layer2)
decoded_layer4 = Dense(21, activation='linear')(decoded_layer3)

autoencoder1 = Model(input_layer1, decoded_layer4)

autoencoder1.compile(optimizer='adam', loss='mse', metrics=[MeanAbsoluteError(), 'mae'])
history1 = autoencoder1.fit(x_train1, x_train1, epochs=1000, batch_size=32, shuffle=True, validation_data=(x_test1, x_test1))

# Create the first Encoder model
encoder1 = Model(input_layer1, encoded_layer4)

# Freeze the first Encoder layers
encoder1.trainable = False

# Create the first pre-trained model
pretrained_model1 = Model(input_layer1, encoder1(input_layer1))

# Define the second Autoencoder
input_layer2 = Input(shape=(3,))
encoded_layer5 = Dense(4, activation='tanh')(input_layer2)
encoded_layer6 = Dense(5, activation='tanh')(encoded_layer5)
encoded_layer7 = Dense(6, activation='tanh')(encoded_layer6)
decoded_layer5 = Dense(5, activation='tanh')(encoded_layer7)
decoded_layer6 = Dense(4, activation='tanh')(decoded_layer5)
decoded_layer7 = Dense(3, activation='tanh')(decoded_layer6)
decoded_layer8 = Dense(1, activation='linear')(decoded_layer7)

autoencoder2 = Model(input_layer2, decoded_layer8)

autoencoder2.compile(optimizer='adam', loss='mse', metrics=[MeanAbsoluteError(), 'mae'])
history2 = autoencoder2.fit(encoder1.predict(x_train1), x_train1, epochs=1000, batch_size=32, shuffle=True, validation_data=(encoder1.predict(x_test1), x_test1))

# Create the second Encoder model
encoder2 = Model(input_layer2, encoded_layer7)

# Freeze the second Encoder layers
encoder2.trainable = False

# Create the pre-trained model
pretrained_model2 = Model(input_layer1, encoder2(encoder1(input_layer1)))

# Combine the two Autoencoders
autoencoder = Model(input_layer1, autoencoder2(encoder1(input_layer1)))

autoencoder.compile(optimizer='adam', loss='mse', metrics=[MeanAbsoluteError(), 'mae','mean_absolute_percentage_error'])
history = autoencoder.fit(x_train1, x_train1, epochs=1000, batch_size=32, shuffle=True, validation_data=(x_test1, x_test1))


In [ ]:
plt.plot(history.history['mean_absolute_error'], label='mean_absolute_error')
plt.plot(history.history['val_mean_absolute_error'], label='Validation_mean_absolute_error')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.35])
plt.legend()
plt.show()


In [ ]:
# Define the Autoencoder architecture
input_layer = Input(shape=(21,))
encoded_layer1 = Dense(6, activation='tanh')(input_layer)
encoded_layer2 = Dense(5, activation='tanh')(encoded_layer1)
encoded_layer3 = Dense(4, activation='tanh')(encoded_layer2)
encoded_layer4 = Dense(3, activation='tanh')(encoded_layer3)
decoded_layer1 = Dense(4, activation='tanh')(encoded_layer4)
decoded_layer2 = Dense(5, activation='tanh')(decoded_layer1)
decoded_layer3 = Dense(6, activation='tanh')(decoded_layer2)
decoded_layer4 = Dense(21, activation='linear')(decoded_layer3)

autoencoder = Model(input_layer, decoded_layer4)

autoencoder.compile(optimizer='adam', loss='mse', metrics=[MeanAbsoluteError(), 'mae'])
history = autoencoder.fit(x_train1, x_train1, epochs=1000, batch_size=32, shuffle=True, validation_data=(x_test1, x_test1))

# Create the Encoder model
encoder = Model(input_layer, encoded_layer4)

# Freeze the Encoder layers
encoder.trainable = False

# Create the pre-trained model
pretrained_model = Model(input_layer, encoder(input_layer))

# Create the final autoencoder
input_layer_final = Input(shape=(21,))
encoded_layer1_final = Dense(6, activation='tanh')(input_layer_final)
encoded_layer2_final = Dense(5, activation='tanh')(encoded_layer1_final)
encoded_layer3_final = Dense(4, activation='tanh')(encoded_layer2_final)
decoded_layer1_final = Dense(4, activation='tanh')(encoded_layer4_final)
encoded_layer4_final = Dense(3, activation='tanh')(encoded_layer3_final)
decoded_layer2_final = Dense(5, activation='tanh')(decoded_layer1_final)
decoded_layer3_final = Dense(6, activation='tanh')(decoded_layer2_final)
decoded_layer4_final = Dense(21, activation='linear')(decoded_layer3_final)

autoencoder_final = Model(input_layer_final, decoded_layer4_final)
autoencoder_final.compile(optimizer='adam', loss='mse', metrics=['mae'])
history_final = autoencoder_final.fit(x_train1, x_train1, epochs=1000, batch_size=32, shuffle=True, validation_data=(x_test1, x_test1))

# Create the Encoder model for the final autoencoder
encoder_final = Model(input_layer_final, encoded_layer4_final)

# Freeze the final autoencoder Encoder layers
encoder_final.trainable = False

# Create the pre-trained model for the final autoencoder
pretrained_model_final = Model(input_layer_final, encoder_final(input_layer_final))

# Extract the hidden layer output of the final autoencoder
hidden_layer_output_final = encoder_final.predict(x_train1)


In [ ]:
plt.plot(history.history['mean_absolute_error'], label='mean_absolute_error')
plt.plot(history.history['val_mean_absolute_error'], label='Validation_mean_absolute_error')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.35])
plt.legend()
plt.show()
